In [42]:
import pandas as pd
import hmmlearn as hmm
import tslearn
from tslearn.clustering import TimeSeriesKMeans
import numpy as np
import matplotlib.pyplot as plt
from tslearn.utils import to_time_series_dataset
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import datetime
import random
random.seed(0)

In [43]:
br_poor = pd.read_csv("SimData/bank_reserves_outputs_poor.csv", header=None)
br_middle = pd.read_csv("SimData/bank_reserves_outputs_middle.csv", header=None)
br_rich = pd.read_csv("SimData/bank_reserves_outputs_rich.csv", header=None)

In [44]:
ecv_active = pd.read_csv("SimData/epsteinCV_outputs_active.csv", header=None)
ecv_jailed = pd.read_csv("SimData/epsteinCV_outputs_jailed.csv", header=None)
ecv_quiet = pd.read_csv("SimData/epsteinCV_outputs_quiet.csv", header=None)

In [45]:
ss_price = pd.read_csv("SimData/sugarscape_outputs_price.csv", header=None)
ss_trader = pd.read_csv("SimData/sugarscape_outputs_trader.csv", header=None)
ss_tradeVol = pd.read_csv("SimData/sugarscape_outputs_tradeVol.csv", header=None)
ss_price.fillna(0, inplace=True)
ss_trader.fillna(0, inplace=True)
ss_tradeVol.fillna(0, inplace=True)

In [46]:
def import_ff_data(filename):
    expected_columns=155
    data = []
    with open(filename, 'r') as file:
        for line in file:
            row = line.strip().split(',')
            if len(row) < expected_columns:
                row += [np.nan] * (expected_columns - len(row))
            data.append(row)
    df = pd.DataFrame(data)
    def fill_last_valid(row):
        for i in range(1, len(row)):
            if pd.isna(row[i]):
                row[i] = row[i - 1]  
        return row
    df_filled = df.apply(fill_last_valid, axis=1)
    return df_filled

In [47]:
ff_onfire = import_ff_data("SimData/forest_fire_outputs_onfire.csv")
print("check 1")
ff_fine = import_ff_data("SimData/forest_fire_outputs_fine.csv")
print("check 2")
ff_burned = import_ff_data("SimData/forest_fire_outputs_burned.csv")

check 1
check 2


# Principal Component Analysis

In [48]:
def pca_reduction(ts0, ts1, ts2):
    sc = StandardScaler()
    pca = PCA(n_components=3)
    PCA_std_list = list()
    for i in np.arange(0, ts0.shape[0]):
        if(i % 2000==0):
            print("On iteration ", i, "at ", datetime.datetime.now())
        sample = pd.concat((ts0.iloc[i], ts1.iloc[i]), axis=1)
        sample = pd.concat((sample, ts2.iloc[i]), axis=1)
        sample = sample.T
        sample_std = sc.fit_transform(sample)
        sample_PCA_std = pca.fit_transform(sample_std)
        
        PCA_std_list.append(pd.Series(sample_PCA_std.flatten())) 
    PCA_std_df = pd.DataFrame(PCA_std_list)
    return PCA_std_df

In [49]:
br_pca = pca_reduction(br_poor, br_middle, br_rich)

On iteration  0 at  2024-11-11 10:56:24.164396
On iteration  2000 at  2024-11-11 10:56:30.578738
On iteration  4000 at  2024-11-11 10:56:37.225559
On iteration  6000 at  2024-11-11 10:56:43.579663
On iteration  8000 at  2024-11-11 10:56:49.786820
On iteration  10000 at  2024-11-11 10:56:55.892537
On iteration  12000 at  2024-11-11 10:57:02.369391
On iteration  14000 at  2024-11-11 10:57:08.554087
On iteration  16000 at  2024-11-11 10:57:14.889929
On iteration  18000 at  2024-11-11 10:57:21.148592
On iteration  20000 at  2024-11-11 10:57:27.361660
On iteration  22000 at  2024-11-11 10:57:33.678687
On iteration  24000 at  2024-11-11 10:57:39.965624
On iteration  26000 at  2024-11-11 10:57:46.396182
On iteration  28000 at  2024-11-11 10:57:52.591055
On iteration  30000 at  2024-11-11 10:57:58.747364
On iteration  32000 at  2024-11-11 10:58:04.992774
On iteration  34000 at  2024-11-11 10:58:11.199930
On iteration  36000 at  2024-11-11 10:58:17.613719
On iteration  38000 at  2024-11-11 10:5

In [50]:
ecv_pca = pca_reduction(ecv_active, ecv_jailed, ecv_quiet)

On iteration  0 at  2024-11-11 11:01:45.608507
On iteration  2000 at  2024-11-11 11:01:54.167343
On iteration  4000 at  2024-11-11 11:02:02.714393
On iteration  6000 at  2024-11-11 11:02:11.299657
On iteration  8000 at  2024-11-11 11:02:20.183886
On iteration  10000 at  2024-11-11 11:02:28.779451
On iteration  12000 at  2024-11-11 11:02:37.377378
On iteration  14000 at  2024-11-11 11:02:46.108566
On iteration  16000 at  2024-11-11 11:02:54.601650
On iteration  18000 at  2024-11-11 11:03:03.009943
On iteration  20000 at  2024-11-11 11:03:11.466522
On iteration  22000 at  2024-11-11 11:03:20.035608
On iteration  24000 at  2024-11-11 11:03:28.607820
On iteration  26000 at  2024-11-11 11:03:37.223378
On iteration  28000 at  2024-11-11 11:03:45.722497
On iteration  30000 at  2024-11-11 11:03:54.222900
On iteration  32000 at  2024-11-11 11:04:02.839728
On iteration  34000 at  2024-11-11 11:04:11.289793
On iteration  36000 at  2024-11-11 11:04:19.557166
On iteration  38000 at  2024-11-11 11:0

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

...

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

On iteration  50000 at  2024-11-10 21:42:58.133919


/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total

In [65]:
#ss_pca = pca_reduction(ss_price, ss_trader, ss_tradeVol)

On iteration  0 at  2024-11-10 21:08:34.051005
On iteration  2000 at  2024-11-10 21:08:44.643433


/Users/maria/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


IndexError: single positional indexer is out-of-bounds

In [51]:
ff_pca = pca_reduction(ff_onfire, ff_fine, ff_burned)

On iteration  0 at  2024-11-11 11:05:21.667108
On iteration  2000 at  2024-11-11 11:05:28.778242
On iteration  4000 at  2024-11-11 11:05:35.898984
On iteration  6000 at  2024-11-11 11:05:43.077198
On iteration  8000 at  2024-11-11 11:05:50.209075
On iteration  10000 at  2024-11-11 11:05:57.171959
On iteration  12000 at  2024-11-11 11:06:04.337925
On iteration  14000 at  2024-11-11 11:06:11.420015


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


On iteration  16000 at  2024-11-11 11:06:18.587084
On iteration  18000 at  2024-11-11 11:06:25.514465
On iteration  20000 at  2024-11-11 11:06:32.425123
On iteration  22000 at  2024-11-11 11:06:39.673114
On iteration  24000 at  2024-11-11 11:06:46.625144
On iteration  26000 at  2024-11-11 11:06:53.733197
On iteration  28000 at  2024-11-11 11:07:00.685927
On iteration  30000 at  2024-11-11 11:07:07.738253
On iteration  32000 at  2024-11-11 11:07:14.719265
On iteration  34000 at  2024-11-11 11:07:21.965223


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


On iteration  36000 at  2024-11-11 11:07:28.947527


...

On iteration  62000 at  2024-11-11 11:09:00.892492
On iteration  64000 at  2024-11-11 11:09:08.330037


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


On iteration  66000 at  2024-11-11 11:09:15.559509
On iteration  68000 at  2024-11-11 11:09:22.532947
On iteration  70000 at  2024-11-11 11:09:29.729041
On iteration  72000 at  2024-11-11 11:09:36.781400
On iteration  74000 at  2024-11-11 11:09:43.785371
On iteration  76000 at  2024-11-11 11:09:50.865978
On iteration  78000 at  2024-11-11 11:09:57.898084


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:652: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


On iteration  80000 at  2024-11-11 11:10:05.134687
On iteration  82000 at  2024-11-11 11:10:12.069648
On iteration  84000 at  2024-11-11 11:10:19.481850
On iteration  86000 at  2024-11-11 11:10:26.508213
On iteration  88000 at  2024-11-11 11:10:33.560343
On iteration  90000 at  2024-11-11 11:10:40.718746
On iteration  92000 at  2024-11-11 11:10:47.676318
On iteration  94000 at  2024-11-11 11:10:54.742634
On iteration  96000 at  2024-11-11 11:11:01.734961
On iteration  98000 at  2024-11-11 11:11:08.810604


In [58]:
br_pca.to_csv('extracted_features/bank_reserves_PCA_standardized.csv')
ecv_pca.to_csv('extracted_features/epstein_PCA_standardized.csv')
#ss_pca.to_csv('/Users/maria/Desktop/TS-Clustering/extracted_features/sugarscape_PCA_standardized.csv')
ff_pca.to_csv('extracted_features/forestfire_PCA_standardized.csv')

In [52]:
ff_pca.shape

(100000, 9)

In [53]:
ecv_pca.shape

(50362, 9)

In [54]:
br_pca.shape

(100000, 9)

# ARMA

In [102]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from pmdarima.arima import auto_arima
import pmdarima as pm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def arima_reduction(ts0, ts1, ts2):
    df_all_arima = pd.DataFrame()
    for i in np.arange(0, ts0.shape[0]):
        if(i % 500 == 0):
            print(i)
        sample_ts0_std = ts0.iloc[i]
        stepwise_fit_ts0 = pm.auto_arima(sample_ts0_std, start_p=1, start_q=1, max_p=3, max_q=3,
                                 trace=False,
                                 error_action='ignore', 
                                 suppress_warnings=True, 
                                 stepwise=True) 
        coefs_ts0 = stepwise_fit_ts0.summary().tables[1]
        html_ts0 = coefs_ts0.as_html()
        df_coefs_ts0 = pd.read_html(html_ts0, header=0)[0]
        df_coefs_small_ts0 = df_coefs_ts0[['Unnamed: 0', 'coef']]
        df_coefs_small_ts0 = df_coefs_small_ts0.set_index('Unnamed: 0')
        df_coefs_small_ts0 = df_coefs_small_ts0.T
        df_coefs_small_ts0 = df_coefs_small_ts0.add_suffix('_ts0')
    
        sample_ts1_std = ts1.iloc[i]
        stepwise_fit_ts1 = pm.auto_arima(sample_ts1_std, start_p=1, start_q=1, max_p=3, max_q=3,
                                 trace=False,
                                 error_action='ignore', 
                                 suppress_warnings=True, 
                                 stepwise=True) 
        coefs_ts1 = stepwise_fit_ts1.summary().tables[1]
        html_ts1 = coefs_ts1.as_html()
        df_coefs_ts1 = pd.read_html(html_ts1, header=0)[0]
        df_coefs_small_ts1 = df_coefs_ts1[['Unnamed: 0', 'coef']]
        df_coefs_small_ts1 = df_coefs_small_ts1.set_index('Unnamed: 0')
        df_coefs_small_ts1 = df_coefs_small_ts1.T
        df_coefs_small_ts1 = df_coefs_small_ts1.add_suffix('_ts1')
    
        sample_ts2_std = ts2.iloc[i]
        stepwise_fit_ts2 = pm.auto_arima(sample_ts2_std, start_p=1, start_q=1, max_p=3, max_q=3,
                                 trace=False,
                                 error_action='ignore', 
                                 suppress_warnings=True, 
                                 stepwise=True) 
        coefs_ts2 = stepwise_fit_ts2.summary().tables[1]
        html_ts2 = coefs_ts2.as_html()
        df_coefs_ts2 = pd.read_html(html_ts2, header=0)[0]
        df_coefs_small_ts2 = df_coefs_ts2[['Unnamed: 0', 'coef']]
        df_coefs_small_ts2 = df_coefs_small_ts2.set_index('Unnamed: 0')
        df_coefs_small_ts2 = df_coefs_small_ts2.T
        df_coefs_small_ts2 = df_coefs_small_ts2.add_suffix('_ts2')
    
        df_coefs_all = pd.concat([df_coefs_small_ts0, df_coefs_small_ts1], axis=1)
        df_coefs_all = pd.concat([df_coefs_all, df_coefs_small_ts2], axis=1)
        df_all_arima = pd.concat([df_all_arima, df_coefs_all], ignore_index=True)
    return df_all_arima

In [ ]:
br_arima = arima_reduction(br_poor, br_middle, br_rich)

0


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = n

500


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = n

1000


...

C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: divide by zero encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: divide by zero encountered in divide
  test_statistic = numer_squared_sum / denom_s

32500


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = n

33000


C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1420: RuntimeWarning: invalid value encountered in divide
  test_statistic = n

In [ ]:
br_arima.to_csv('extracted_features/bank_reserves_arima.csv')

In [ ]:
ecv_arima = arima_reduction(ecv_active, ecv_jailed, ecv_quiet)

In [ ]:
ecv_arima.to_csv('extracted_features/epstein_arima.csv')

In [ ]:
ff_arima = arima_reduction(ff_onfire, ff_fine, ff_burned)

In [ ]:
ff_arima.to_csv('extracted_features/ff_arima.csv')

# Hidden Markov Model

# Fast Fourier Transform

In [20]:
sc = StandardScaler()
bank_reserves_HMM = pd.DataFrame()
bank_reserves_HMM_std = pd.DataFrame()
#for i in np.arange(0, br_poor.shape[0]):
sample = pd.concat((br_poor.iloc[0], br_middle.iloc[0]), axis=1)
sample = pd.concat((sample, br_rich.iloc[0]), axis=1)
sample_std = sc.fit_transform(sample)
sample_HMM = np.fft.fft(sample)
sample_HMM_std = np.fft.fft(sample_std)
bank_reserves_HMM = pd.concat([bank_reserves_HMM, (pd.Series(sample_HMM[0:9].flatten()))], axis=1)
bank_reserves_HMM_std = pd.concat([bank_reserves_HMM_std, (pd.Series(sample_HMM_std[0:9].flatten()))], axis=1)

In [24]:
sample_HMM

array([[ 500.   +0.j        , -250. -433.01270189j, -250. +433.01270189j],
       [ 476.   +0.j        , -238. -325.62555182j, -238. +325.62555182j],
       [ 475.   +0.j        , -237.5-295.31466269j, -237.5+295.31466269j],
       [ 477.   +0.j        , -237. -245.95121467j, -237. +245.95121467j],
       [ 461.   +0.j        , -226. -219.97045256j, -226. +219.97045256j],
       [ 475.   +0.j        , -230. -206.1140461j , -230. +206.1140461j ],
       [ 472.   +0.j        , -222.5-172.33905535j, -222.5+172.33905535j],
       [ 479.   +0.j        , -226. -183.5973856j , -226. +183.5973856j ],
       [ 478.   +0.j        , -219.5-163.67880132j, -219.5+163.67880132j],
       [ 475.   +0.j        , -207.5-129.03778516j, -207.5+129.03778516j],
       [ 470.   +0.j        , -202. -126.43970895j, -202. +126.43970895j],
       [ 479.   +0.j        , -203.5-132.50188678j, -203.5+132.50188678j],
       [ 482.   +0.j        , -196. -126.43970895j, -196. +126.43970895j],
       [ 483.   +0.j     